In [2]:
import pandas as pd

from google.colab import files
uploaded = files.upload()

import io
pre_covid_df = pd.read_csv(io.BytesIO(uploaded['disinfecting_wipe_Pre_Covid.csv']))

Saving disinfecting_wipe_Pre_Covid.csv to disinfecting_wipe_Pre_Covid.csv


In [3]:
pre_covid_df.head()

,product_category,product_url,review_date,Date,handle,rating,helpfulness_rating,review
0,disinfecting_wipes,https://www.amazon.com/dp/B00Q70RCW6,2/22/2018,Pre_Covid,K Lilledahl,1,One,*****EVERY SINGLE TIME I ORDER THESE THEY COME...
1,disinfecting_wipes,https://www.amazon.com/dp/B00Q70RCW6,11/11/2019,Pre_Covid,andychen278,5,NaN,5 stars
2,disinfecting_wipes,https://www.amazon.com/dp/B00Q70RCW6,6/2/2018,Pre_Covid,Rachel Anschuetz,5,NaN,"80 wipes lasts forever, and they work. they sm..."
3,disinfecting_wipes,https://www.amazon.com/dp/B00Q70RCW6,10/27/2018,Pre_Covid,Mohabee Serrano,5,NaN,"A great product, and it's a bit cheaper than t..."
4,disinfecting_wipes,https://www.amazon.com/dp/B00Q70RCW6,7/21/2019,Pre_Covid,Robint,4,NaN,"A little concerned that when package arrived, ..."


In [4]:
pre_covid_df = pre_covid_df.drop(columns=['product_url', 'helpfulness_rating'], axis=1)

In [5]:
pre_covid_df['rating'] = pre_covid_df['rating'].apply(lambda z:'positive' if z>3 else 'negative')

In [6]:
pre_covid_df.head()

,product_category,review_date,Date,handle,rating,review
0,disinfecting_wipes,2/22/2018,Pre_Covid,K Lilledahl,negative,*****EVERY SINGLE TIME I ORDER THESE THEY COME...
1,disinfecting_wipes,11/11/2019,Pre_Covid,andychen278,positive,5 stars
2,disinfecting_wipes,6/2/2018,Pre_Covid,Rachel Anschuetz,positive,"80 wipes lasts forever, and they work. they sm..."
3,disinfecting_wipes,10/27/2018,Pre_Covid,Mohabee Serrano,positive,"A great product, and it's a bit cheaper than t..."
4,disinfecting_wipes,7/21/2019,Pre_Covid,Robint,positive,"A little concerned that when package arrived, ..."


In [7]:
import nltk

nltk.download('punkt')

import string
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
pre_covid_df['review'] = pre_covid_df['review'].astype(str)

In [9]:
from nltk.tokenize import word_tokenize

for item in pre_covid_df['review']:
  words = word_tokenize(item.replace('\n',' '))

In [10]:
clean = [word for word in words if word not in set(string.punctuation)]

In [11]:
characters_to_remove = ["''",'``','!',"'"]
clean_words = [word for word in clean if word not in set(characters_to_remove)]

In [12]:
nltk.download('stopwords')

from nltk.corpus import stopwords
english_stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
clean = [word for word in clean_words if word not in english_stops]

In [14]:
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [15]:
reviews = [(list(clean), rating)
              for review_text in pre_covid_df['review']
              for rating in pre_covid_df['rating']]

random.shuffle(reviews)

In [16]:
clean = [wordnet_lemmatizer.lemmatize(word) for word in clean]

In [17]:
all_words = nltk.FreqDist(word.lower() for word in clean)
word_features = list(all_words)

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [18]:
split = 0.7

featuresets = [(document_features(d), c) for (d,c) in reviews]

train_set, test_set = featuresets[int(round(len(featuresets)*split,0))-1:], featuresets[:int(round(len(featuresets)*split,0))-1]

nb_classifier = nltk.NaiveBayesClassifier.train(train_set)

In [19]:
print(nltk.classify.accuracy(nb_classifier, test_set))

0.8089939591311602


In [22]:
nb_classifier.show_most_informative_features(n=15)

Most Informative Features
            contains(go) = True           negati : positi =      1.0 : 1.0
            contains(ca) = True           negati : positi =      1.0 : 1.0
         contains(every) = True           negati : positi =      1.0 : 1.0
         contains(lysol) = False          negati : positi =      1.0 : 1.0
       contains(perfect) = False          negati : positi =      1.0 : 1.0
          contains(will) = False          negati : positi =      1.0 : 1.0
           contains(buy) = True           negati : positi =      1.0 : 1.0
           contains(you) = False          negati : positi =      1.0 : 1.0
        contains(always) = True           negati : positi =      1.0 : 1.0
          contains(time) = True           negati : positi =      1.0 : 1.0
           contains(say) = False          negati : positi =      1.0 : 1.0
         contains(wrong) = True           negati : positi =      1.0 : 1.0
           contains(n't) = True           negati : positi =      1.0 : 1.0